In [1]:
import face_recognition
import cv2
import numpy as np


In [10]:
video_capture = cv2.VideoCapture(0)


In [2]:
priya_image = face_recognition.load_image_file("F:\projects\Crowed_Monetering\photos_spp\Priyanav\IMG-20240413-WA0021.jpg")
priya_face_encoding = face_recognition.face_encodings(priya_image)[0]

In [3]:
shivam_image = face_recognition.load_image_file("F:\projects\Crowed_Monetering\photos_spp\Shivam\IMG-20240415-WA0011.jpg")
shivam_face_encoding = face_recognition.face_encodings(shivam_image)[0]
gupta_image = face_recognition.load_image_file("F:\projects\Crowed_Monetering\photos_spp\Priyanshu\WhatsApp Image 2024-04-15 at 00.29.16_c0234071.jpg")
gupta_face_encoding = face_recognition.face_encodings(gupta_image)[0]
unkur_image = face_recognition.load_image_file("unkur.jpg")
unkur_face_encoding = face_recognition.face_encodings(unkur_image)[0]


In [4]:
known_face_encodings = [
    priya_face_encoding,
    shivam_face_encoding,
    gupta_face_encoding,
    unkur_face_encoding
]

In [5]:
known_face_names = [
    "priyanav ",
    "Shivam",
    "gupta",
    "Ankur"
]

In [6]:
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

In [11]:
while True:
  
    ret, frame = video_capture.read()

    
    if process_this_frame:
       
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

       
        rgb_small_frame = small_frame[:, :, ::-1]
        
        
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            face_names.append(name)

    process_this_frame = not process_this_frame


    
    for (top, right, bottom, left), name in zip(face_locations, face_names):
       
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    
    cv2.imshow('Video', frame)

    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


video_capture.release()
cv2.destroyAllWindows()

In [ ]:
# import a utility function for loading Roboflow models
from inference import get_model
# import supervision to visualize our results
import supervision as sv
# import cv2 to helo load our image
import cv2
# define the image url to use for inference

image_file = "image.jpg"
image = cv2.imread(image_file)

# load a pre-trained yolov8n model
model = get_model(model_id="bike-helmet-detection-2vdjo/2")

# run inference on our chosen image, image can be a url, a numpy array, a PIL image, etc.
results = model.infer(image)

# load the results into the supervision Detections api
detections = sv.Detections.from_inference(results[0].dict(by_alias=True, exclude_none=True))

# create supervision annotators
bounding_box_annotator = sv.BoundingBoxAnnotator()
label_annotator = sv.LabelAnnotator()

# annotate the image with our inference results
annotated_image = bounding_box_annotator.annotate(
    scene=image, detections=detections)
annotated_image = label_annotator.annotate(
    scene=annotated_image, detections=detections)

# display the image
sv.plot_image(annotated_image)